<a href="https://colab.research.google.com/github/Praveen-Prabhu/code-generation/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 1 - SMTSolvers - Output

In [ ]:
import subprocess
from typing import Tuple

class SMTSolvers:
    def __init__(self, timeout: int = 15, debug: bool = False):
        """
        Initialize the SMTSolvers object with configuration parameters.

        Args:
            timeout: Maximum execution time in seconds for tested code
            debug: Whether to print debug information
        """
        self.timeout = timeout
        self.debug = debug

    def verify(self, code: str, test_code: str) -> Tuple[float, str]:
        """
        Verifies generated code by running it against provided test code.

        Args:
            code: The generated solution code to test
            test_code: The test code to validate the solution

        Returns:
            Tuple containing:
                - reward score (float): 2.0 for passing tests, 0.0 for failing, -0.5 for timeout
                - output (str): execution output or error message
        """
        try:
            # Clean up the test code if it's malformed
            if isinstance(test_code, list) or (isinstance(test_code, str) and "," in test_code and len(test_code) < 100):
                return 0.0, "Warning: Test code appears to be malformed"

            # Combine the generated solution with the test code
            full_code = f"{code}\n\n{test_code}"

            if self.debug:
                print(f"Running code:\n{full_code[:200]}...")

            # Create a temporary file to execute
            with open("temp_test.py", "w") as f:
                f.write(full_code)

            # Run the test file
            result = subprocess.run(
                ["python3", "temp_test.py"],
                capture_output=True, text=True, timeout=self.timeout
            )

            # Check if execution was successful (no errors)
            if result.returncode == 0:
                if self.debug:
                    print("Test passed successfully!")
                return 2.0, result.stdout  # Increased reward for correct execution
            else:
                if self.debug:
                    print(f"Test failed with error: {result.stderr}")
                return 0.0, result.stderr  # Test failed

        except subprocess.TimeoutExpired:
            if self.debug:
                print("Execution timed out")
            return -0.5, "Execution timed out"  # Penalize timeouts more severely

        except Exception as e:
            if self.debug:
                print(f"Execution error: {str(e)}")
            return 0.0, str(e)  # Test failed

solver = SMTSolvers(debug=True)

### ✅ Logically Correct Code

In [ ]:
code = """
from typing import List

def below_zero(operations: List[int]) -> bool:
    # Initialize the balance to 0
    balance = 0

    # Iterate over each operation in the list
    for operation in operations:
        # Add the operation to the balance
        balance += operation

        # If the balance is less than 0, return True immediately
        if balance < 0:
            return True

    # If we've iterated over all operations and the balance is non-negative, return False
    return False
"""

test_code = """
def check(candidate):
    assert candidate('(()()) ((())) () ((())()())') == [
        '(()())', '((()))', '()', '((())()())'
    ]
    assert candidate('() (()) ((())) (((())))') == [
        '()', '(())', '((()))', '(((())))'
    ]
    assert candidate('(()(())((())))') == [
        '(()(())((())))'
    ]
    assert candidate('( ) (( )) (( )( ))') == ['()', '(())', '(()())']
"""
reward, output = solver.verify(code, test_code)
print(f"Reward: {reward}")

### ❌ Logically Incorrect Code

In [ ]:
code = """
def below_zero(operations: List[int]) -> bool:
    # Initialize the balance to zero
    balance = 0

    # Iterate over each operation in the list
    for operation in operations:
        # Add the operation to the balance
        balance += operation

        # If the balance falls below zero, return True
        if balance < 0:
            return True

    # If the function hasn't returned True after iterating over all operations, return False
    return False
"""

test_code = """
def check(candidate):
    assert candidate([]) == False
    assert candidate([1, 2, -3, 1, 2, -3]) == False
    assert candidate([1, 2, -4, 5, 6]) == True
    assert candidate([1, -1, 2, -2, 5, -5, 4, -4]) == False
    assert candidate([1, -1, 2, -2, 5, -5, 4, -5]) == True
    assert candidate([1, -2, 2, -2, 5, -5, 4, -4]) == True
"""

reward, output = solver.verify(code, test_code)
print(f"Reward: {reward}")

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

model_url: https://huggingface.co/PraveenPrabhu/codegen-llama-3.1-8b-grpo-merged-16bit

branch: main

In [ ]:
#@title 2. Launch the web UI

#@markdown If unsure about the branch, write "main" or leave it blank.

import os
from pathlib import Path

os.environ.pop('PYTHONPATH', None)
os.environ.pop('MPLBACKEND', None)

if Path.cwd().name != 'text-generation-webui':
  print("\033[1;32;1m\n --> Installing the web UI. This will take a while, but after the initial setup, you can download and test as many models as you like.\033[0;37;0m\n")

  !git clone https://github.com/oobabooga/text-generation-webui
  %cd text-generation-webui

  # Install the project in an isolated environment
  !GPU_CHOICE=A \
  USE_CUDA118=FALSE \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh

# Parameters
model_url = "https://huggingface.co/PraveenPrabhu/codegen-llama-3.1-8b-grpo-merged-16bit" #@param {type:"string"}
branch = "main" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant --no_flash_attn" #@param {type:"string"}
api = True #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

!$cmd